#**SISTEMA DE RECOMENDACIÓN CON SVD**

In [ ]:
# Incluye aquí todas las librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from google.colab import drive
import os

# Google Colab Mount
drive.mount('/content/drive')
DIR = "/content/drive/MyDrive/IA"
os.chdir(DIR)

# Si se desean comentar algunos de los Warnings.
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga el archivo "restaurant+consumer+data.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [ ]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)

(1161, 5) (130, 21)


**NOTA: Recuerda que cada vez que se te pida incluir líneas de código, podrás incluir 1 o más líneas, pero siempre en la sección donde se te indica hacerlo.**

## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

#### El próposito del argumento latin-1 es proporcionar un conjunto de caracteres de 8 bits que se utilizan para representar texto. Este argumento incluye letras mayúsculas, minúsculas y con acentos. En la norma ISO el próposito de este argumento es permitir la amplia gama en la codificación en diferentes idiomas.

In [ ]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [ ]:
# Del segundo archivo obtenemos información diversa de cada restaurante:
data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [ ]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [ ]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":
df_combinado = pd.merge(data1a, data2a, on='placeID')
# Despleguemos la dimensión y los primeros renglones de este DataFrame:
print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [ ]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".
UtMx_food = df_combinado.pivot_table(values='food_rating',  index='name', columns='userID', fill_value=0 )
print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [ ]:
# a. Aplica la transformación SVD para obtener la matriz de las variables
#    latentes de los restaurantes, en relación a la manera en que los usuarios
#    evaluaron la comida. Deberás obtener la factorización SVD con el máximo número
#    de componentes y por lo tanto el máximo de valores singulares posibles de acuerdo
#    a este criterio. Así, de acuerdo al resultado obtenido en el Ejercicio-3, asigna
#    a la variable "nc_food" el número entero adecuado para llegar a obtener el
#    resultado pedido.

nc_food =UtMx_food.shape[1] - 1
print('Total de valores singulares basados en evaluación de la comida:', nc_food)

Total de valores singulares basados en evaluación de la comida: 137


In [ ]:
# b. Lleva a cabo la factorización SVD truncada en la matriz de utilidad
#    obtenida previamente y con respecto al número de componentes dada
#    por la variable nc_food.

SVD_food = TruncatedSVD(n_components=nc_food, random_state=1)
SVD_food.fit(UtMx_food)

# Con la factorización SVD truncada obtenida, determinemos la cantidad
# de componentes que explican un 90% de la variabilidad acumulada de
# cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1
print('Valor de truncamiento al 90% de dicha variabilidad:', N_food)

Valor de truncamiento al 90% de dicha variabilidad: 51


In [ ]:
# Usando la métrica de correlación de Pearson, obtenemos ahora las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similitudes con el "Restaurante Pueblo Bonito"
# y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtenemos las 10 mejores correlaciones positivas a continuación.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:5]

Total de similaridades positivas encontradas: 72
Algunos de los resultados encontrados:


[(0.005365772268925685, 'Arrachela Grill'),
 (0.0237030560854367, 'Cabana Huasteca'),
 (0.03203479494263902, 'Cafe Chaires'),
 (0.01818890428904317, 'Cafeteria cenidet'),
 (0.22439383669651536, 'Cafeteria y Restaurant El Pacifico')]

## **Ejercicio - 5**

In [ ]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente. Llamaremos "mejores_sim_food_ordenadas" a
# dicha lista.
mejores_sim_food_ordenadas = sorted(mejores_sim_food,reverse=True)
# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (0.7889806551020476, 'Restaurante la Estrella de Dima')
2> (0.7027625945811559, 'pizza clasica')
3> (0.6694220289110384, 'Restaurante Guerra')
4> (0.5422148774570532, 'El Club')
5> (0.3686961464811633, 'Pizzeria Julios')
6> (0.31162901041144253, 'Restaurant Oriental Express')
7> (0.30827094702272917, 'Hamburguesas saul')
8> (0.307803206076139, 'Restaurante El Cielo Potosino')
9> (0.28239278769439985, 'Rincon Huasteco')
10> (0.26861901399389676, 'la Cochinita Pibil Restaurante Yucateco')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación de un restaurante con respecto a su servicio (service_rating), encuentra ahora los diez restaurantes con mayor similaridad al mismo restaurante anterior llamado "Restaurante Pueblo Bonito" y con respecto también a la misma métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [ ]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".
UtMx_service = df_combinado.pivot_table(values='service_rating', index='name', columns='userID', fill_value=0)

print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

# ************* Inlcuye aquí tu código:*****************************
explained_variance_ratio_threshold = 0.9
SVD_service = TruncatedSVD(n_components=UtMx_service.shape[1])
resultant_matrix_service = SVD_service.fit_transform(UtMx_service)

# Calcula el número de componentes necesario para explicar el 90% de la varianza acumulada
cumulative_variance_ratio = np.cumsum(SVD_service.explained_variance_ratio_)
num_components_to_reach_threshold = np.argmax(cumulative_variance_ratio >= explained_variance_ratio_threshold)

# Actualiza el número de componentes para alcanzar el umbral del 90% de varianza explicada
SVD_service = TruncatedSVD(n_components=num_components_to_reach_threshold)
resultant_matrix_service = SVD_service.fit_transform(UtMx_service)

# Calcula la matriz de correlación de Pearson
corr_mat_service = np.corrcoef(resultant_matrix_service)

# Restaurante de referencia
restaurante_de_referencia_service = "Restaurante Pueblo Bonito"
nombres_rest_service = UtMx_service.index  # nombres de restaurantes
idx_rest_service = list(nombres_rest_service).index(restaurante_de_referencia_service)

# Correlaciones con el restaurante de referencia
corr_rest_service = corr_mat_service[idx_rest_service]

# Encuentra las 10 mejores correlaciones positivas (sin incluir el restaurante de referencia)
idx_service = (corr_rest_service > 0)
mejores_sim_service = [(corr_rest_service[i], nombres_rest_service[i]) for i in range(len(nombres_rest_service)) if i != idx_rest_service and idx_service[i]]
mejores_sim_service.sort(reverse=True)
mejores_sim_service_10 = mejores_sim_service[:10]

# Resultados
print('\nLos 10 restaurantes con mayor similitud de Pearson al "Restaurante Pueblo Bonito" en términos de servicio:')
for i, (similitud, restaurante) in enumerate(mejores_sim_service_10, 1):
    print(f"{i}> Restaurante: {restaurante}, Similitud: {similitud}")

# *********** Aquí termina la sección de agregar código *************


Los 10 restaurantes con mayor similitud de Pearson al "Restaurante Pueblo Bonito" en términos de servicio:
1> Restaurante: pizza clasica, Similitud: 0.6653498754728695
2> Restaurante: El Club, Similitud: 0.5843103622458914
3> Restaurante: Hamburguesas saul, Similitud: 0.4975742031712366
4> Restaurante: Restaurante la Estrella de Dima, Similitud: 0.4688863494412881
5> Restaurante: Restaurante Guerra, Similitud: 0.43072832192603455
6> Restaurante: Restaurant Orizatlan, Similitud: 0.40101781770411493
7> Restaurante: Restaurante El Cielo Potosino, Similitud: 0.3840916567424661
8> Restaurante: la Cochinita Pibil Restaurante Yucateco, Similitud: 0.35490616727838187
9> Restaurante: La Fontana Pizza Restaurante and Cafe, Similitud: 0.3345773486589678
10> Restaurante: Restaurant Oriental Express, Similitud: 0.3265693831159911


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

En el ejercicio 5 podemos visualizar que los dos restaurantes con correlaciones altas para la calificación de la comida tenemos al Restaurante El cielo Potosino y la Hamburguesería Saul con una correlación ambos de 0.30.

**Restaurantes con Correlaciones Altas**: Algunos de los restaurantes que mostraron correlaciones altas tanto para la calificación de la comida como del servicio incluyen:
Claro, aquí hay una descripción más detallada de los restaurantes que mostraron correlaciones altas tanto para la calificación de la comida como del servicio:

* Restaurantes Más Similares: El Rest "Pizza Clasica", mostro similitudes tanto en términos de calidad de la comida como del servicio, lo que sugiere que estos establecimientos ofrecen una experiencia gastronómica integralmente positiva para los usuarios.

* Resultados de Similitud Por Calidad de Comida: Los restaurantes más similares incluyeron "Restaurante la Estrella de Dima", "Pizza Clasica" y "Restaurante Guerra", con correlaciones positivas significativas.

* Resultados de Similitud Por Calidad de Servicio: Los restaurantes más similares incluyeron "Pizza Clasica", "El Club" y "Hamburguesas Saul", con correlaciones positivas significativas.

El SVD podemos usarla para descomponer una matriz de covarianza, la cual nos permite desarrollar un algoritmo para un sistema de recomendaciones.
